You just got hired as the first and only data practitioner at a small business experiencing exponential growth. The company needs more structured processes, guidelines, and standards. Your first mission is to structure the human resources data. The data is currently scattered across teams and files and comes in various formats: Excel files, CSVs, JSON files...

You'll work with the following data in the `datasets` folder:
- __Office addresses__
    - Saved in `office_addresses.csv`. 
    - If the value for office is `NaN`, then the employee is remote.
- __Employee addresses__
    - Saved on the first tab of `employee_information.xlsx`.
- __Employee emergency contacts__ 
    - Saved on the second tab of `employee_information.xlsx`; this tab is called `emergency_contacts`. 
    - However, this sheet was edited at some point, and ***the headers were removed***! The HR manager let you know that they should be: `employee_id`, `last_name`, `first_name`, `emergency_contact`, `emergency_contact_number`, and `relationship`.
- __Employee roles, teams, and salaries__ 
    - This information has been exported from the company's human resources management system into a JSON file titled `employee_roles.json`. Here are the first few lines of that file:
```
{"A2R5H9":
  {
    "title": "CEO",
    "monthly_salary": "$4500",
    "team": "Leadership"
  },
 ...
}
```

Read in, merge, and clean the four datasets to make a single combined pandas DataFrame.

Create a single pandas DataFrame called employees_final containing:
Index: employee_id,
Columns: employee_first_name, employee_last_name, employee_country, employee_city, employee_street, employee_street_number, emergency_contact, emergency_contact_number, relationship, monthly_salary, team, title, office, office_country, office_city, office_street, office_street_number.
Change any missing values in column names starting with office to the word "Remote".

In [2]:
import pandas as pd 
import openpyxl

# Specify the path to your Excel file
xlsx_file_path = 'datasets/employee_information.xlsx'

# Open the Excel workbook
workbook = openpyxl.load_workbook(xlsx_file_path)

# Get the names of all sheets in the workbook
sheet_names = workbook.sheetnames

# Print the sheet names
print("Sheet Names:")
for sheet_name in sheet_names:
    print(sheet_name)

# Close the workbook
workbook.close()

Sheet Names:
employee_addresses
emergency_contacts


In [3]:
office_addresses = pd.read_csv('datasets/office_addresses.csv')

employee_addresses = pd.read_excel('datasets/employee_information.xlsx', sheet_name='employee_addresses')

In [4]:
header = ['employee_id', 'last_name', 'first_name', 'emergency_contact', 'emergency_contact_number', 'relationship']

emergency_contacts = pd.read_excel('datasets/employee_information.xlsx', sheet_name='emergency_contacts')

emergency_contacts.columns = header

In [5]:
employee_roles = pd.read_json('datasets/employee_roles.json')

In [41]:
office_addresses = office_addresses[['office', 'office_country', 'office_city',
                                     'office_street', 'office_street_number']]

In [42]:
office_addresses.head()

,office,office_country,office_city,office_street,office_street_number
0,Leuven Office,BE,Leuven,Martelarenlaan,38
1,ESB Office,US,New York City,Fifth Avenue,350
2,WeWork Office,GB,London,Old Street,207


In [22]:
employee_addresses = employee_addresses[['employee_id', 'employee_first_name', 'employee_last_name', 'employee_country', 'employee_city', 'employee_street', 'employee_street_number']]

In [23]:
employee_addresses.head()

,employee_id,employee_first_name,employee_last_name,employee_country,employee_city,employee_street,employee_street_number
0,A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9
1,H8K0L6,Tara,Siff,GB,London,Baker Street,221
2,G4R7V0,Gemma,Sagal,US,New-York,Perry Street,66
3,M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7


In [24]:
emergency_contacts = emergency_contacts[['emergency_contact', 'emergency_contact_number', 'relationship']]

In [25]:
emergency_contacts.head()

,emergency_contact,emergency_contact_number,relationship
0,Wendy de Matteo,+44-020-5554-333,Sister
1,John Newmark,+1-202-555-194,Husband
2,Venus Noone,+1-202-555-0130,Wife


In [30]:
employee_roles = employee_roles.transpose()

In [35]:
employee_roles = employee_roles[['monthly_salary', 'team', 'title']]

In [37]:
employee_roles.head()

,monthly_salary,team,title
A2R5H9,$4500,Leadership,CEO
H8K0L6,$4500,Leadership,CFO
G4R7V0,$3000,Sales,Business Developer
M1Z7U9,$2000,People Operations,Office Manager


In [56]:
employees_final = pd.concat([employee_addresses,emergency_contacts,
                       employee_roles, office_addresses,], ignore_index=True)

In [57]:
# Assuming employees_final is your DataFrame
# Replace 'employee_id' with the actual column name you want to use as the index
employees_final.set_index('employee_id', inplace=True)

In [58]:
# Assuming employees_final is your DataFrame
# Replace 'office_' with the actual prefix of your column names
columns_to_replace = [col for col in employees_final.columns if col.startswith('office')]

# Replace missing values in selected columns with 'Remote'
employees_final[columns_to_replace] = employees_final[columns_to_replace].fillna('Remote')

In [59]:
employees_final.head()

,employee_first_name,employee_last_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,monthly_salary,team,title,office,office_country,office_city,office_street,office_street_number
employee_id,,,,,,,,,,,,,,,,,
A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9.0,NaN,NaN,NaN,NaN,NaN,NaN,Remote,Remote,Remote,Remote,Remote
H8K0L6,Tara,Siff,GB,London,Baker Street,221.0,NaN,NaN,NaN,NaN,NaN,NaN,Remote,Remote,Remote,Remote,Remote
G4R7V0,Gemma,Sagal,US,New-York,Perry Street,66.0,NaN,NaN,NaN,NaN,NaN,NaN,Remote,Remote,Remote,Remote,Remote
M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7.0,NaN,NaN,NaN,NaN,NaN,NaN,Remote,Remote,Remote,Remote,Remote
NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wendy de Matteo,+44-020-5554-333,Sister,NaN,NaN,NaN,Remote,Remote,Remote,Remote,Remote


In [ ]:
# Import pandas
import pandas as pd

# Read in office_addresses.csv
offices = pd.read_csv("datasets/office_addresses.csv")

# Read in employee_information.xlsx
addresses = pd.read_excel("datasets/employee_information.xlsx")

# Declare a list of new column names
emergency_contacts_header = ["employee_id", "last_name", "first_name",
                             "emergency_contact", "emergency_contact_number", "relationship"]

# Read in employee_information.xlsx
emergency_contacts = pd.read_excel("datasets/employee_information.xlsx", 
                                   sheet_name="emergency_contacts", 
                                   header=None,
                                   names=emergency_contacts_header)

# Read in employee_roles.json
roles = pd.read_json("datasets/employee_roles.json", orient="index")

# Merge addresses with offices
employees = addresses.merge(offices, left_on="employee_country", right_on="office_country", how="left")

# Merge employees with roles
employees = employees.merge(roles, left_on="employee_id", right_on=roles.index)

# Merge employees with emergency_contacts
employees = employees.merge(emergency_contacts, on="employee_id")

# Fill null values in office columns
for col in ["office", "office_country", "office_city", "office_street", "office_street_number"]:
    employees[col].fillna("Remote", inplace=True)
    
# Create final columns
final_columns = ["employee_id", "employee_first_name", "employee_last_name", "employee_country", 
                 "employee_city", "employee_street", "employee_street_number", 
                 "emergency_contact", "emergency_contact_number", "relationship", 
                 "monthly_salary", "team", "title", "office", "office_country", 
                 "office_city", "office_street", "office_street_number"]

# Subset for the required columns
employees_final = employees[final_columns]

# Set employee_id as the index
employees_final.set_index("employee_id", inplace=True)
employees_final.head()